### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()


,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Calculate the total number of schools
total_schools = len(school_data_complete['school_name'].unique())

#Calculate the total number of students
total_students = school_data_complete['student_name'].count()

#Calculate the total budget
total_budget = sum(school_data_complete['budget'].unique())

#Calculate the average maths score
average_maths_score = school_data_complete['maths_score'].mean()

#Calculate the average reading score
average_reading_score = school_data_complete['reading_score'].mean()

#Calculate the percentage of students with a passing maths score (50 or greater)
pass_maths_percent = ((school_data_complete[school_data_complete['maths_score'] >= 50].maths_score.count()) / total_students)*100

#Calculate the percentage of students with a passing reading score (50 or greater)
pass_reading_percent = ((school_data_complete[school_data_complete['reading_score'] >= 50].reading_score.count()) / total_students)*100

# #Calculate the percentage of students who passed maths and reading (% Overall Passing)
overall_pass_percent = school_data_complete[(school_data_complete['maths_score'] >= 50) & (school_data_complete['reading_score'] >= 50)]['student_name'].count()/total_students*100

#Create a dataframe to hold the above results & format numbers
summary_data = {"Total Schools":[total_schools], "Total Students":[total_students], "Total Budget":[total_budget], "Average Maths Score":[average_maths_score], "Average Reading Score":[average_reading_score], "% Passing Maths":[pass_maths_percent], "% Passing Reading":[pass_reading_percent], "% Overall Passing":[overall_pass_percent]}
local_govt_summary = pd.DataFrame(summary_data)
local_govt_summary.head().style.format({"Total Students": "{:,.0f}", "Total Budget": "${:,.0f}", "Average Maths Score": "{:.2f}", "Average Reading Score": "{:.2f}", "% Passing Maths": "{:.2f}%", "% Passing Reading": "{:.2f}%", "% Overall Passing": "{:.2f}%"})


,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",70.34,69.98,86.08%,84.43%,72.81%


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
school_type = school_data.set_index(['school_name'])['type']
school_total_students = school_data_complete['school_name'].value_counts()
school_total_budget = school_data.set_index(['school_name'])['budget']
school_per_student_budget = school_total_budget / school_total_students

#Group and calculate mean
grouped_average_score = school_data_complete.groupby(['school_name']).agg({'reading_score': 'mean', 'maths_score': 'mean'})
school_average_maths_score = grouped_average_score['maths_score']
school_average_reading_score = grouped_average_score['reading_score']

#Group and calculate maths and reading scores
passing_maths_score = school_data_complete.loc[school_data_complete['maths_score'] >= 50]
grouped_passing_maths_score = passing_maths_score.groupby(['school_name']).agg({'student_name': 'count'})
school_pass_maths_percent = (grouped_passing_maths_score['student_name']/school_total_students)*100


passing_reading_score = school_data_complete.loc[school_data_complete['reading_score'] >= 50]
grouped_passing_reading_score = passing_reading_score.groupby(['school_name']).agg({'student_name': 'count'})
school_pass_reading_percent = (grouped_passing_reading_score['student_name']/school_total_students)*100

#overall pass percent
passing_overall_score = school_data_complete[(school_data_complete['maths_score'] >= 50) & (school_data_complete['reading_score'] >= 50)]
grouped_passing_overall_score = passing_overall_score.groupby(['school_name']).agg({'student_name': 'count'})
school_overall_pass_percent = (grouped_passing_overall_score['student_name']/school_total_students)*100                                                                                 

# Create a dataframe to hold the above results
school_summary = {"School Type": school_type, "Total Students": school_total_students, "Total School Budget": school_total_budget, "Per Student Budget": school_per_student_budget, "Average Maths Score": school_average_maths_score, "Average Reading Score": school_average_reading_score, "% Passing Maths": school_pass_maths_percent, "% Passing Reading": school_pass_reading_percent, "% Overall Passing": school_overall_pass_percent}
school_summary_data = pd.DataFrame(school_summary)
school_summary_data.index.name = None
school_summary_data.style.format({"Total Students": "{:,.0f}", "Total School Budget": "${:,.0f}", "Per Student Budget": "${:,.2f}", "Average Maths Score": "{:.2f}", "Average Reading Score": "{:.2f}", "% Passing Maths": "{:.2f}%", "% Passing Reading": "{:.2f}%", "% Overall Passing": "{:.2f}%"})


,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Bailey High School,Government,"4,976","$3,124,928",$628.00,72.35,71.01,91.64%,87.38%,80.08%
Cabrera High School,Independent,"1,858","$1,081,356",$582.00,71.66,71.36,90.85%,89.07%,80.79%
Figueroa High School,Government,"2,949","$1,884,411",$639.00,68.70,69.08,81.65%,82.81%,67.65%
Ford High School,Government,"2,739","$1,763,916",$644.00,69.09,69.57,82.44%,82.22%,67.47%
Griffin High School,Independent,"1,468","$917,500",$625.00,71.79,71.25,91.21%,88.49%,81.34%
Hernandez High School,Government,"4,635","$3,022,020",$652.00,68.87,69.19,80.95%,81.88%,66.36%
Holden High School,Independent,427,"$248,087",$581.00,72.58,71.66,89.93%,88.52%,78.92%
Huang High School,Government,"2,917","$1,910,635",$655.00,68.94,68.91,81.69%,81.45%,66.71%
Johnson High School,Government,"4,761","$3,094,650",$650.00,68.84,69.04,82.06%,81.98%,67.19%
Pena High School,Independent,962,"$585,858",$609.00,72.09,71.61,91.68%,86.59%,79.21%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
top_schools = school_summary_data.sort_values(by=['% Overall Passing'], ascending=False)
top_schools.head(5).style.format({"Total Students": "{:,.0f}", "Total School Budget": "${:,.0f}", "Per Student Budget": "${:,.2f}", "Average Maths Score": "{:.2f}", "Average Reading Score": "{:.2f}", "% Passing Maths": "{:.2f}%", "% Passing Reading": "{:.2f}%", "% Overall Passing": "{:.2f}%"})


,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Griffin High School,Independent,"1,468","$917,500",$625.00,71.79,71.25,91.21%,88.49%,81.34%
Cabrera High School,Independent,"1,858","$1,081,356",$582.00,71.66,71.36,90.85%,89.07%,80.79%
Bailey High School,Government,"4,976","$3,124,928",$628.00,72.35,71.01,91.64%,87.38%,80.08%
Wright High School,Independent,"1,800","$1,049,400",$583.00,72.05,70.97,91.78%,86.67%,79.72%
Rodriguez High School,Government,"3,999","$2,547,363",$637.00,72.05,70.94,90.80%,87.40%,79.42%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
bottom_schools = school_summary_data.sort_values(by=['% Overall Passing'], ascending=True)
bottom_schools.head(5).style.format({"Total Students": "{:,.0f}", "Total School Budget": "${:,.0f}", "Per Student Budget": "${:,.2f}", "Average Maths Score": "{:.2f}", "Average Reading Score": "{:.2f}", "% Passing Maths": "{:.2f}%", "% Passing Reading": "{:.2f}%", "% Overall Passing": "{:.2f}%"})


,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Hernandez High School,Government,"4,635","$3,022,020",$652.00,68.87,69.19,80.95%,81.88%,66.36%
Huang High School,Government,"2,917","$1,910,635",$655.00,68.94,68.91,81.69%,81.45%,66.71%
Johnson High School,Government,"4,761","$3,094,650",$650.00,68.84,69.04,82.06%,81.98%,67.19%
Wilson High School,Independent,"2,283","$1,319,574",$578.00,69.17,68.88,82.79%,81.30%,67.46%
Ford High School,Government,"2,739","$1,763,916",$644.00,69.09,69.57,82.44%,82.22%,67.47%


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
year_9 = school_data_complete.loc[student_data['year'] == 9].groupby('school_name')["maths_score"].mean()
year_10 = school_data_complete.loc[student_data['year'] == 10].groupby('school_name')["maths_score"].mean()
year_11 = school_data_complete.loc[student_data['year'] == 11].groupby('school_name')["maths_score"].mean()
year_12 = school_data_complete.loc[student_data['year'] == 12].groupby('school_name')["maths_score"].mean()

maths_score_summary = {"Year 9": year_9, "Year 10": year_10, "Year 11": year_11, "Year 12": year_12}
maths_score_year = pd.DataFrame(maths_score_summary)
maths_score_year.index.name = None
maths_score_year.style.format({"Year 9": "{:,.2f}", "Year 10": "{:,.2f}", "Year 11": "{:,.2f}", "Year 12": "{:.2f}"})

,Year 9,Year 10,Year 11,Year 12
Bailey High School,72.49,71.90,72.37,72.68
Cabrera High School,72.32,72.44,71.01,70.60
Figueroa High School,68.48,68.33,68.81,69.33
Ford High School,69.02,69.39,69.25,68.62
Griffin High School,72.79,71.09,71.69,71.47
Hernandez High School,68.59,68.87,69.15,68.99
Holden High School,70.54,75.11,71.64,73.41
Huang High School,69.08,68.53,69.43,68.64
Johnson High School,69.47,67.99,68.64,69.29
Pena High School,72.00,72.40,72.52,71.19


## Reading Score by Year

* Perform the same operations as above for reading scores

In [7]:
year_9r = school_data_complete.loc[student_data['year'] == 9].groupby('school_name')["reading_score"].mean()
year_10r = school_data_complete.loc[student_data['year'] == 10].groupby('school_name')["reading_score"].mean()
year_11r = school_data_complete.loc[student_data['year'] == 11].groupby('school_name')["reading_score"].mean()
year_12r = school_data_complete.loc[student_data['year'] == 12].groupby('school_name')["reading_score"].mean()

reading_score_summary = {"Year 9": year_9r, "Year 10": year_10r, "Year 11": year_11r, "Year 12": year_12r}
reading_score_year = pd.DataFrame(reading_score_summary)
reading_score_year.index.name = None
reading_score_year.style.format({"Year 9": "{:,.2f}", "Year 10": "{:,.2f}", "Year 11": "{:,.2f}", "Year 12": "{:.2f}"})

,Year 9,Year 10,Year 11,Year 12
Bailey High School,70.90,70.85,70.32,72.20
Cabrera High School,71.17,71.33,71.20,71.86
Figueroa High School,70.26,67.68,69.15,69.08
Ford High School,69.62,68.99,70.74,68.85
Griffin High School,72.03,70.75,72.39,69.43
Hernandez High School,68.48,70.62,68.42,69.24
Holden High School,71.60,71.10,73.31,70.48
Huang High School,68.67,69.52,68.74,68.67
Johnson High School,68.72,69.30,69.97,67.99
Pena High School,70.95,72.32,71.70,71.51


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
bins = [0, 584, 629, 644, 675]

group_name = ["$<585", "$585-630", "$630-645", "$645-680"]

school_summary_data["spend_bins"] = pd.cut(school_summary_data["Total School Budget"]/school_summary_data["Total Students"], bins, labels = group_name)

group_spend_bins = school_summary_data.groupby("spend_bins")

average_maths = group_spend_bins['Average Maths Score'].mean()

average_reading = group_spend_bins['Average Reading Score'].mean()

ave_maths_percent = group_spend_bins['% Passing Maths'].mean()

ave_reading_percent = group_spend_bins['% Passing Reading'].mean()

ave_overall_percent = group_spend_bins['% Overall Passing'].mean()

group_bin = {"Average Maths Score": average_maths, "Average Reading Score": average_reading, "% Passing Maths": ave_maths_percent, "% Passing Reading": ave_reading_percent, "% Overall Passing": ave_overall_percent}
group_bin_summary = pd.DataFrame(group_bin)
group_bin_summary.index.name = "Spending Ranges (Per Student)"
group_bin_summary.style.format({"Average Maths Score": "{:,.2f}", "Average Reading Score": "{:,.2f}", "% Passing Maths": "{:,.2f}", "% Passing Reading": "{:.2f}", "% Overall Passing": "{:.2f}"})


,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
$<585,71.36,70.72,88.84,86.39,76.72
$585-630,72.07,71.03,91.52,87.29,79.88
$630-645,69.85,69.84,84.69,83.76,71.00
$645-680,68.88,69.05,81.57,81.77,66.76


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
sbins = [0, 999, 1999, 4999]

sgroup_name = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_data["size_bins"] = pd.cut(school_data["size"], sbins, labels = sgroup_name)

school_data.rename(columns={"budget":"Total School Budget"}, inplace=True)

new_sch_summary = pd.merge(school_data, school_summary_data, on='Total School Budget')

group_size_bins = new_sch_summary.groupby("size_bins")

sch_ave_maths = group_size_bins['Average Maths Score'].mean()

sch_ave_reading = group_size_bins['Average Reading Score'].mean()

sch_ave_maths_percent = group_size_bins['% Passing Maths'].mean()

sch_ave_reading_percent = group_size_bins['% Passing Reading'].mean()

sch_ave_overall_percent = group_size_bins['% Overall Passing'].mean()

sch_group_bin = {"Average Maths Score": sch_ave_maths, "Average Reading Score": sch_ave_reading, "% Passing Maths": sch_ave_maths_percent, "% Passing Reading": sch_ave_reading_percent, "% Overall Passing": sch_ave_overall_percent}
sch_group_bin_summary = pd.DataFrame(sch_group_bin)
sch_group_bin_summary.index.name = "School Size"
sch_group_bin_summary.style.format({"Average Maths Score": "{:,.2f}", "Average Reading Score": "{:,.2f}", "% Passing Maths": "{:,.2f}", "% Passing Reading": "{:.2f}", "% Overall Passing": "{:.2f}"})


,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.34,71.64,90.81,87.56,79.07
Medium (1000-2000),71.42,70.72,89.85,86.71,78.04
Large (2000-5000),69.75,69.58,84.25,83.30,70.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [10]:
group_type_bins = school_summary_data[["School Type", "Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]]
group_type_bins = group_type_bins.groupby("School Type")
group_type_bins.mean()


,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
